# Домашнее задание 1.
Тема: Морфология

## Задание
Обработать текст книги с помощью морфологического анализатора, сделать поверхностный анализ.

### 1. Выбрать и сохранить книгу в .txt. Проследите, что кодировка UTF-8

In [1]:
with open('garri_potter.txt', 'r') as infile:
    text = infile.read().replace('Annotation', '')

annotation = text[:280].strip()
print(annotation)

Книга, покорившая мир, эталон литературы, синоним успеха. Книга, сделавшая Дж.К. Роулинг самым читаемым писателем современности. Книга, ставшая культовой уже для нескольких поколений. «Гарри Поттер и философский камень» — история начинается. Для среднего школьного возраста.


### 2. Обработать книгу с помощью mystem:

In [15]:
from pymystem3 import Mystem

stem = Mystem()

* лемматизировать с помощью mystem

In [3]:
text_lemmatize = stem.lemmatize(text)

* сохранить результат в файл

In [4]:
text_lemmatize_join = ''.join(text_lemmatize)

with open('garri_potter_pymystem.txt', 'w') as outfile:
    outfile.write(text_lemmatize_join)

* не на оценку, опционально: замерить время работы на примере небольшого кусочка текста (например, глава) (как замерить время см. ниже)

In [154]:
%%timeit
m.lemmatize(annotation)

1.48 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 3. Обработать книгу через pymorphy

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

In [6]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

* токенизировать текст с помощью nltk (см. конспект)

In [7]:
text_tokenize = word_tokenize(text)

# токенизируем текст, приводим к нижнему регистру и оставляем только последовательности из букв
text_tokenize = [w.lower() for w in text_tokenize if w.isalpha()]

# фильтруем стопслова
sw = stopwords.words('russian')
text_tokenize = [w for w in text_tokenize if w not in sw]

* разобрать слова с помощью pymorphy

In [8]:
text_tokenize_morph = [morph.parse(w) for w in text_tokenize]

* сохраните результат в jsonlines: каждая строчка - это разбор слова в виде словаря {"lemma": "конь", "word": "коня", "pos": "NOUN"}

In [9]:
jsonlines = ''
for parse in text_tokenize_morph:
    jsonlines += str({"lemma": parse[0].normal_form, "word": parse[0].word, "pos": parse[0].tag.POS}) + '\n'

In [10]:
with open('garri_potter_pymorphy.jsonl', 'w') as infile:
    infile.write(jsonlines)

* не на оценку, опционально: замерить время работы (как с майстемом)

In [162]:
%%timeit
word_tokenize(annotation)

385 µs ± 9.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 4. Ответить на вопросы:

In [11]:
import collections

* Какую долю слов составляет каждая часть речи? (Например, для глагола - это количество глаголов, деленное на общее число слов в тексте)

In [12]:
text_tokenize_morph_pos = [x[0].tag.POS for x in text_tokenize_morph]

In [13]:
all_count = len(text_tokenize_morph_pos)
pos_count = collections.Counter(text_tokenize_morph_pos)
for pos, count in pos_count.items():
    print(pos, count/all_count)

NOUN 0.41768412246071795
PRTF 0.008366231796909724
ADJF 0.10815969044942352
NUMR 0.007895631258333551
VERB 0.2546994692671704
ADJS 0.00988261131009961
PRCL 0.01911161076106565
ADVB 0.07806740045491385
PRTS 0.002980470077649089
INTJ 0.003163481398206489
INFN 0.03626238594473058
CONJ 0.010222489476849068
COMP 0.006588407540066407
GRND 0.015634395670475046
NPRO 0.012653925592825957
PREP 0.007268163873565322
PRED 0.0010980679233444013
None 0.0002614447436534289


* Найдите топ-20 (по частотности) глаголов и наречий (стоп слова можно убирать, можно и не убирать)

In [28]:
text_tokenize_morph_verb = [x[0].normal_form for x in text_tokenize_morph if x[0].tag.POS == 'VERB' or x[0].tag.POS == 'INFN']
verb_count = collections.Counter(text_tokenize_morph_verb)
verb_count_sorted = sorted(verb_count.items(), key=lambda item: item[1])
print(verb_count_sorted[:-20:-1])

[('сказать', 282), ('знать', 176), ('злеть', 131), ('дудлить', 127), ('видеть', 112), ('спросить', 107), ('хотеть', 99), ('ответить', 98), ('говорить', 93), ('мочь', 92), ('смотреть', 81), ('стать', 70), ('идти', 69), ('стоять', 56), ('дурслить', 56), ('посмотреть', 55), ('заметить', 55), ('уизлить', 54), ('быть', 54)]


In [20]:
text_tokenize_morph_advb = [x[0].normal_form for x in text_tokenize_morph if x[0].tag.POS == 'ADVB']
advb_count = collections.Counter(text_tokenize_morph_advb)
advb_count_sorted = sorted(advb_count.items(), key=lambda item: item[1])
print(advb_count_sorted[:-20:-1])

[('ещё', 182), ('очень', 91), ('пока', 76), ('едва', 52), ('снова', 49), ('быстро', 38), ('затем', 37), ('сегодня', 35), ('рядом', 33), ('тихо', 32), ('сразу', 32), ('несколько', 32), ('почему', 30), ('вместе', 29), ('прямо', 29), ('громко', 28), ('вообще', 27), ('точно', 27), ('сюда', 26)]


### 5. Посмотрите документацию для nltk н-грамм...

Посмотрите документацию для nltk н-грамм (nltk.bigrams, например), попробуйте составить топ-25 биграмм и триграмм для вашего текста в лемматизированном виде (только леммы, без знаков препинания).

In [23]:
text_tokenize_lemmatize = stem.lemmatize(' '.join(text_tokenize))
text_tokenize_lemmatize = list(filter(lambda x: x != ' ' and x != '\n', text_tokenize_lemmatize))

In [24]:
text_tokenize_lemmatize_bigrams = list(nltk.bigrams(text_tokenize_lemmatize))
bigrams_count = collections.Counter(text_tokenize_lemmatize_bigrams)
bigrams_count_sorted = sorted(bigrams_count.items(), key=lambda item: item[1])
print(bigrams_count_sorted[:-25:-1])

[(('дядя', 'вернон'), 104), (('профессор', 'макгонаголла'), 92), (('тетя', 'петуния'), 52), (('сказать', 'гарри'), 44), (('гарри', 'рон'), 39), (('волшебный', 'палочка'), 33), (('рон', 'гермиона'), 28), (('сказать', 'рон'), 27), (('спрашивать', 'гарри'), 27), (('гарри', 'поттер'), 27), (('отвечать', 'гарри'), 26), (('мистер', 'дурсло'), 26), (('гарри', 'гермиона'), 23), (('профессор', 'страунс'), 22), (('рон', 'гарри'), 20), (('миссис', 'дурслей'), 19), (('профессор', 'думбльдор'), 17), (('все', 'равный'), 17), (('сказать', 'огрид'), 16), (('друг', 'друг'), 16), (('гарри', 'знать'), 16), (('конец', 'конец'), 16), (('бирючинный', 'улица'), 16), (('миссис', 'норрис'), 15)]


In [25]:
text_tokenize_lemmatize_trigrams = list(nltk.trigrams(text_tokenize_lemmatize))
trigrams_count = collections.Counter(text_tokenize_lemmatize_trigrams)
trigrams_count_sorted = sorted(trigrams_count.items(), key=lambda item: item[1])
print(trigrams_count_sorted[:-25:-1])

[(('платформа', 'девять', 'четверть'), 8), (('гарри', 'рон', 'гермиона'), 6), (('сказать', 'дядя', 'вернон'), 6), (('коридор', 'третий', 'этаж'), 5), (('дядя', 'вернон', 'тетя'), 5), (('проходить', 'мимо', 'пушка'), 4), (('портрет', 'толстой', 'тетя'), 4), (('квидиш', 'сквозь', 'век'), 4), (('фред', 'джордж', 'уизли'), 4), (('гарри', 'покачать', 'голова'), 4), (('платформа', 'девять', 'десять'), 4), (('колдовство', 'ведьминский', 'искусство'), 4), (('школа', 'колдовство', 'ведьминский'), 4), (('вернон', 'тетя', 'петуния'), 4), (('гарри', 'дядя', 'вернон'), 4), (('сказать', 'профессор', 'макгонаголла'), 4), (('марс', 'сегодня', 'яркий'), 3), (('сто', 'пятьдесят', 'балл'), 3), (('потерять', 'дар', 'речь'), 3), (('профессор', 'макгонаголла', 'гарри'), 3), (('малфа', 'крабба', 'гойл'), 3), (('защита', 'сила', 'зло'), 3), (('гарри', 'профессор', 'макгонаголла'), 3), (('орешек', 'берти', 'ботт'), 3)]


Почему получаются именно такие?

Основной причиной образования N-грамм является часто встречающаяся последовательность этих слов в тексте. Персонажи в одной сцене ('гарри', 'рон', 'гермиона'), описательные слова ('школа', 'колдовство', 'ведьминский'), стереотипные повадки ('гарри', 'покачать', 'голова') используюся в одном контексте чаще всего, как очень любимые автором.

### 6. Возьмите абзац из изначального текста...

Возьмите абзац из изначального текста (3 - 8 предложений), замените в нём время всех глаголов (например, прошедшее на настоящее или будущее), число всех существительных (единственное на множественное, множественное на единственное), сделайте согласование по числу глаголов и существительных. Например, вместо изначального Слон подарил мартышке цветы должно получиться Слоны подарят мартышкам цветок.

In [167]:
def changer(word):
    w = word[0]
    cases = list(w.tag.grammemes)[1:]
    cases = list(map(lambda x: x.replace('sing', 'plur') if ('VERB' in cases or 'INFN' in cases or 'NOUN' in cases) and 'sing' in cases else x.replace('plur', 'sing'), cases))
    cases = list(map(lambda x: x.replace('past', 'futr') if 'VERB' in cases and 'past' in cases else x.replace('futr', 'past'), cases))
    if 'VERB' in cases:
        cases.append('3per')
    w = w.inflect(set(cases))
    if not w:
        w = word[0]
    return w.word

In [168]:
source_text = 'Слон подарил мартышке цветы'
source_text_morph = [morph.parse(w) for w in source_text.split()]
changed_words = list(map(changer, source_text_morph))
changed_text = ' '.join(changed_words).capitalize()
print(source_text)
print(changed_text)

Слон подарил мартышке цветы
Слоны подарят мартышкам цветок


In [187]:
source_text = annotation
source_text_morph = [morph.parse(w) for w in source_text.split()]
changed_words = list(map(changer, source_text_morph))
changed_text = ' '.join(changed_words).capitalize()
print(source_text)
print(changed_text)

Книга, покорившая мир, эталон литературы, синоним успеха. Книга, сделавшая Дж.К. Роулинг самым читаемым писателем современности. Книга, ставшая культовой уже для нескольких поколений. «Гарри Поттер и философский камень» — история начинается. Для среднего школьного возраста.
Книга, покорившая мир, эталоны литературы, синонимы успеха. книга, сделавшая дж.к. роулинг самым читаемым писателями современности. книга, ставшая культовой уже для нескольких поколений. «гарри поттер и философский камень» — истории начинается. для средних школьного возраста.


Далеко не все слова знакомы алгоритму pymorch, поэтому качество оставляет желать лучшего.